##Check song  profile to make sure song ids match song ids in MSD
'caue the track ids don't seem to carry over  
(They do, yay! Well, except for the known mismatches, probably. Or at least I'm going to assume there are no other mismatches until I discover otherwise)  
```
http://developer.echonest.com/api/v4/song/profile?api_key=API_KEY&format=json&id=SOEBCBI12AF72A154F
```

In [1]:
import sys
import requests
import time
import pickle
from pprint import pprint

In [2]:
def pickle_stuff(filename, data):
    ''' save file '''
    with open(filename, 'w') as picklefile:
        pickle.dump(data, picklefile)

def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [7]:
with open("echonestapi1.txt", 'r') as ena:
    api_key = ena.read().strip()

In [8]:
rate_time = 0.501 #3.1

In [3]:
r = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+'&id=musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d')

In [4]:
rdata = r.json()
pprint(rdata)

{u'response': {u'status': {u'code': 0,
                           u'message': u'Success',
                           u'version': u'4.2'},
               u'track': {u'artist': u'Hammock',
                          u'catalog': u'musicbrainz',
                          u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
                          u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
                          u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
                          u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
                          u'id': u'TRRYMSJ13B7BF722BF',
                          u'release': u'More Dead Than Alive (Get Away From the Medicine)',
                          u'song_id': u'SOCGKJR12A58A79D30',
                          u'status': u'complete',
                          u'title': u'I Can Almost See You'}}}


In [5]:
pprint(rdata["response"]["status"]["message"])

u'Success'


In [8]:
pprint(rdata["response"]["track"])
pprint(rdata["response"]["track"]["song_id"])

{u'artist': u'Hammock',
 u'catalog': u'musicbrainz',
 u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
 u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
 u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
 u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
 u'id': u'TRRYMSJ13B7BF722BF',
 u'release': u'More Dead Than Alive (Get Away From the Medicine)',
 u'song_id': u'SOCGKJR12A58A79D30',
 u'status': u'complete',
 u'title': u'I Can Almost See You'}
u'SOCGKJR12A58A79D30'


In [111]:
mbid_songs = {}

In [112]:
with open("mbid_songs.csv", 'r') as msf:
    for line in msf:
        line = line.strip()
        segs = line.split(",")
        key = segs[0]
        track = segs[1]
        mbid_songs[key] = track

In [113]:
print len(mbid_songs.keys())

450466


In [ ]:
linecount = 0
with open("mbids_1.txt", 'r') as mf:
    t0 = time.time()
    for line in mf:
        linecount += 1
        print len(line.split())
        t1 = time.time()
        count = 0
        for mbid in line.split():
            if mbid in mbid_songs:
                #print "continuing"
                count += 1
                continue
            print count,
            count += 1
            moveon = False
            while not moveon:
                #print "requesting"
                time.sleep(rate_time)
                try:
                    enr = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+
                                       '&id=musicbrainz:track:'+mbid)
                    enrj = enr.json()
                    status = enrj["response"]["status"]["message"]
                    if status == "Success":
                        #print "true"
                        if "song_id" in enrj["response"]["track"]:
                            #print "success" #"ids", mbid
                            mbid_songs[mbid] = enrj["response"]["track"]["song_id"]
                        moveon = True
                    elif status == "The Identifier specified does not exist":
                        #echo nest does not have all mbids mapped to tracks
                        #print status #mbid, status
                        mbid_songs[mbid] = ""
                        moveon = True
                    else:
                        print status
                        time.sleep(3)
                except requests.exceptions.ConnectionError as e:
                    print e,
                    time.sleep(3)
        t2 = time.time()
        if t2-t1 > 0.5:
            print "\nline seconds:", t2-t1,
            print "total seconds:", t2-t0, 
            print "total lines:", linecount     


1108
1700
410
926
1535
964
1454
1741
571
1826
858
540
1431
746
2166
1249
1360
1277
1359
1499
1407
1093
643
423
376
2369
1308
1122
856
780
1378
630
987
902
939
1987
1412
564
948
989
1283
914
739
761
661
486
1377
1169
1055
1487
1077
1365
1089
658
874
569
635
2766
282
179
821
2439
953
585
542
1322
577
226
312
613
798
824
545
687
2872
657
1027
981
1272
904
670
1299
192
1252
1006
1120
1393
782
2551
1354
1639
337
1862
348
486
2306
1558
588
1360
1612
857
361
682
988
1040
1368
1921
927
242
34
876
1386
1790
1650
541
1191
1069
756
211
945
1961
1095
1742
1449
2344
1517
1290
717
2304
2190
1461
775
1194
56
1143
14
1687
1081
1891
1545
846
1278
1659
639
880
1397
895
760
338
513
1100
626
378
824
1217
1065
1323
557
2549
477
386
551
410
998
1301
2982
1266
558
2
1678
884
1813
605
1357
665
1660
1974
884
1468
1248
1819
954
1542
1661
196
512
1907
36
219
758
996
1078
549
113
1199
979
711
508
1383
1758
1324
1695
1162
500
65
835
2133
781
38
979
357
1067
337
1010
917
1462
1304
2024
1303
1018
1204
918
568
1170
3

In [116]:
#print mbid_songs

In [108]:
print len(mbid_songs.keys())

 445360


In [109]:
pickle_stuff("mbid_song_dict.pkl", mbid_songs)

In [110]:
with open("mbid_songs.csv", 'w') as msf:
    for k, v in mbid_songs.iteritems():
        msf.write(k+","+v+"\n")